In [1]:
import argparse
import requests
from bs4 import BeautifulSoup
import time
import re
import hashlib
from collections import defaultdict, Counter
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import random
from urllib.parse import urljoin

In [2]:
#conda update scipy

In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tsakunelson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urljoin
import time

class NgiemboonDictionaryScraper:
    def __init__(self, base_url):
        self.base_url = base_url
        self.domain = self.extract_domain(base_url)
        self.crawled_pages = set()
        self.to_crawl = []
        self.dictionary_data = []

    def extract_domain(self, url):
        """Extract the domain from a URL."""
        match = re.match(r'(https?://)?(www\d?\.)?(?P<domain>[\w.-]+\.\w+)', url)
        return match.group('domain') if match else None

    def fetch_page(self, url):
        """Fetch the HTML content of a page."""
        try:
            response = requests.get(url, timeout=5, headers={'User-Agent': 'Mozilla/5.0'})
            if response.status_code == 200:
                return response.text
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
        return None

    def extract_links(self, html):
        """Extract all links from a page."""
        soup = BeautifulSoup(html, 'html.parser')
        links = [a['href'] for a in soup.find_all('a', href=True)]
        return links

    def canonicalize_url(self, url):
        """Convert a relative URL to an absolute URL."""
        return urljoin(self.base_url, url)

    def is_within_browse_directory(self, url):
        """Check if the URL is within the /browse/ directory."""
        return "/browse/" in url

    def extract_dictionary_data(self, html):
        """Extract dictionary entries from a page."""
        soup = BeautifulSoup(html, 'html.parser')
        entries = []

        # Find all dictionary entries
        for entry in soup.select('.reversalindexentry'):
            # Extract the French or English word (first occurrence of lang="fr" or lang="en")
            current_language = None
            word = None
            ngiemboon_word = None
            example_usage = []

            # Extract the word based on the first occurrence of lang="fr" or lang="en"
            for lang_tag in entry.find_all(['span', 'a'], attrs={'lang': True}):
                if lang_tag['lang'] == 'fr':
                    current_language = "French"
                    word = lang_tag.text.strip()
                    break
                elif lang_tag['lang'] == 'en':
                    current_language = "English"
                    word = lang_tag.text.strip()
                    break

            # Extract Ngiemboon words and example usage
            for lang_tag in entry.find_all(['span', 'a'], attrs={'lang': 'nnh'}):
                ngiemboon_word = lang_tag.text.strip()
                example_usage.append(ngiemboon_word)

            # Combine example usage into a single string
            example_usage = " ".join(example_usage)

            if current_language and word and ngiemboon_word:
                entries.append({
                    "Current Language Category": current_language,
                    "word": word,
                    "Ngienmboon Language word": ngiemboon_word,
                    "Example Usage": example_usage
                })

        return entries

    def crawl(self, letter, language):
        """Crawl the website for a specific letter and language."""
        start_url = f"{self.base_url}/browse/{language}/?letter={letter}&lang=en"
        self.to_crawl = [start_url]

        while self.to_crawl:
            url = self.to_crawl.pop(0)
            if url in self.crawled_pages:
                continue
            print(f"Crawling: {url}")
            html = self.fetch_page(url)
            if not html:
                continue
            self.crawled_pages.add(url)

            # Extract dictionary data from the page
            entries = self.extract_dictionary_data(html)
            self.dictionary_data.extend(entries)

            # Extract and add new links to the crawl list (for pagination)
            soup = BeautifulSoup(html, 'html.parser')
            pagination_links = soup.select('#wp_page_numbers a')
            for link in pagination_links:
                full_link = self.canonicalize_url(link['href'])
                if self.is_within_browse_directory(full_link) and full_link not in self.crawled_pages:
                    self.to_crawl.append(full_link)

            time.sleep(1)  # Politeness delay

    def save_to_json(self, filename):
        """Save the extracted data to a JSON file."""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.dictionary_data, f, ensure_ascii=False, indent=4)
        print(f"Data saved to {filename}")

# Main execution
if __name__ == "__main__":
    base_url = "https://www.webonary.org/ngiemboon"
    scraper = NgiemboonDictionaryScraper(base_url)

    # Define letters and languages to crawl
    letters = "abcdefghijklmnopqrstuvwxyz"
    languages = ["francais", "english", "browse-vernacular-english"]

    # Crawl for each letter and language
    for letter in letters:
        for language in languages:
            print(f"Crawling letter '{letter}' for language '{language}'...")
            scraper.crawl(letter, language)

    # Save the collected data to a JSON file
    scraper.save_to_json("ngiemboon_dictionary_full.json")

Crawling letter 'a' for language 'francais'...
Crawling: https://www.webonary.org/ngiemboon/browse/francais/?letter=a&lang=en
Crawling letter 'a' for language 'english'...
Crawling: https://www.webonary.org/ngiemboon/browse/english/?letter=a&lang=en
Crawling letter 'a' for language 'browse-vernacular-english'...
Crawling: https://www.webonary.org/ngiemboon/browse/browse-vernacular-english/?letter=a&lang=en
Crawling letter 'b' for language 'francais'...
Crawling: https://www.webonary.org/ngiemboon/browse/francais/?letter=b&lang=en
Crawling letter 'b' for language 'english'...
Crawling: https://www.webonary.org/ngiemboon/browse/english/?letter=b&lang=en
Crawling letter 'b' for language 'browse-vernacular-english'...
Crawling: https://www.webonary.org/ngiemboon/browse/browse-vernacular-english/?letter=b&lang=en
Crawling letter 'c' for language 'francais'...
Crawling: https://www.webonary.org/ngiemboon/browse/francais/?letter=c&lang=en
Crawling letter 'c' for language 'english'...
Crawling

In [11]:
import json

def explore_json_output(file_path, num_entries=5):
    """
    Explore and display the contents of a JSON file.

    Args:
        file_path (str): Path to the JSON file.
        num_entries (int): Number of entries to display. Default is 5.
    """
    try:
        # Load the JSON file
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Print the total number of entries
        print(f"Total entries in the JSON file: {len(data)}")

        # Display the first few entries
        print("\nFirst few entries:")
        for i, entry in enumerate(data[:num_entries], 1):
            print(f"\nEntry {i}:")
            for key, value in entry.items():
                print(f"{key}: {value}")

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except json.JSONDecodeError:
        print(f"Error: The file '{file_path}' is not a valid JSON file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
if __name__ == "__main__":
    json_file_path = "ngiemboon_dictionary_full.json"  # Replace with your JSON file path
    explore_json_output(json_file_path, num_entries=5)

Total entries in the JSON file: 548

First few entries:

Entry 1:
Current Language Category: French
word: à
Ngienmboon Language word: lé
Example Usage: lé- á1 á né2 né mbwó pà1 pà lé1 lé

Entry 2:
Current Language Category: French
word: à (+verbe)
Ngienmboon Language word: é-
Example Usage: ḿ-1 ḿ- ń-1 ń- é-1 é-

Entry 3:
Current Language Category: French
word: abaisser
Ngienmboon Language word: ńkwɔgte
Example Usage: ńkwɔg1 ńkwɔg ńkwɔgte1 ńkwɔgte

Entry 4:
Current Language Category: French
word: abandon
Ngienmboon Language word: tém nzèm
Example Usage: tém nzèm

Entry 5:
Current Language Category: French
word: abandonner
Ngienmboon Language word: ńkyé
Example Usage: ńnyé2 ńnyé ńkyé


In [12]:
def find_entry(french_word):
    for entry in data:
        if entry["word"].lower() == french_word.lower():
            return entry
    return "Word not found"

In [16]:
try:
    # Load the JSON file
    with open("ngiemboon_dictionary_full.json", 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Print the total number of entries
    print(f"Total entries in the JSON file: {len(data)}")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: The file '{file_path}' is not a valid JSON file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Total entries in the JSON file: 548


In [34]:
# Example usage
french_word = "balance"
result = find_entry(french_word)
print(result)

Word not found


In [46]:
import requests
from bs4 import BeautifulSoup
import json
import time
import string

# Dictionary mapping directories to language categories
LANGUAGES = {
    "francais": "French",
    "browse-english": "English",
    "browse-vernacular-english": "Ngiemboon"
}

BASE_URL_FIRST_PAGE = "https://www.webonary.org/ngiemboon/browse/{directory}/?letter={letter}&key=fr&lang=en"
BASE_URL_NEXT_PAGES = "https://www.webonary.org/ngiemboon/browse/{directory}/?letter={letter}&key=fr&totalEntries={total_entries}&pagenr={page}&lang=en"

def get_total_pages_and_entries(directory, letter):
    """Get total number of pages and total entries for a letter in a directory."""
    url = BASE_URL_FIRST_PAGE.format(directory=directory, letter=letter)
    response = requests.get(url)

    if response.status_code != 200:
        print(f"❌ Failed to retrieve {url}")
        return 1, 0  # Default to 1 page, 0 total entries if failed

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract total pages from pagination info
    page_info = soup.find("li", class_="page_info")
    total_pages = 1  # Default to 1

    if page_info:
        try:
            total_pages = int(page_info.text.strip().split("of")[-1].strip())
        except ValueError:
            pass  # Default to 1 if parsing fails

    # Extract totalEntries from page numbers
    last_page_link = soup.find("li", class_="space")
    total_entries = 0

    if last_page_link:
        try:
            total_entries = int(last_page_link.find_previous_sibling("li").a["href"].split("totalEntries=")[1].split("&")[0])
        except (AttributeError, IndexError, TypeError):
            pass  # Default to 0 if parsing fails

    return total_pages, total_entries

def scrape_page(directory, letter, page, total_entries):
    """Scrape a single page for a given letter and directory."""
    if page == 1:
        url = BASE_URL_FIRST_PAGE.format(directory=directory, letter=letter)
    else:
        url = BASE_URL_NEXT_PAGES.format(directory=directory, letter=letter, total_entries=total_entries, page=page)

    response = requests.get(url)

    if response.status_code != 200:
        print(f"❌ Failed to retrieve page {page} for letter {letter} in {directory}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    entries = []

    for entry in soup.find_all("div", class_="reversalindexentry"):
        french_word_tag = entry.find("span", lang="fr")
        ngiemboon_tags = entry.find_all("span", lang="nnh")
        pos_tag = entry.find("span", lang="en")  # Part of speech

        if french_word_tag and ngiemboon_tags:
            french_word = french_word_tag.text.strip()
            ngiemboon_words = [n.text.strip() for n in ngiemboon_tags]
            pos = pos_tag.text.strip() if pos_tag else ""

            entries.append({
                "Current Language Category": LANGUAGES[directory],
                "word": french_word,
                "Ngienmboon Language word": ngiemboon_words[0],  # First occurrence
                "Example Usage": "".join(ngiemboon_words) + pos
            })

    return entries

def scrape_all_directories():
    """Scrape all letters (A-Z) for each directory and page."""
    all_entries = []
    
    for directory, language_category in LANGUAGES.items():
        print(f"📖 Scraping category: {language_category}")

        for letter in string.ascii_lowercase:  # Loop through a-z
            total_pages, total_entries = get_total_pages_and_entries(directory, letter)
            print(f"  🔤 Scraping letter: {letter} ({total_pages} pages, {total_entries} entries)")

            for page in range(1, total_pages + 1):
                print(f"    📄 Scraping page {page} of {total_pages}")
                entries = scrape_page(directory, letter, page, total_entries)
                all_entries.extend(entries)
                time.sleep(1)  # Delay to avoid getting blocked

    with open("ngiemboon_dictionary.json", "w", encoding="utf-8") as f:
        json.dump(all_entries, f, ensure_ascii=False, indent=4)

    print("✅ Scraping completed. Data saved to ngiemboon_dictionary.json")

scrape_all_directories()


📖 Scraping category: French
  🔤 Scraping letter: a (24 pages, 584 entries)
    📄 Scraping page 1 of 24
    📄 Scraping page 2 of 24
    📄 Scraping page 3 of 24
    📄 Scraping page 4 of 24
    📄 Scraping page 5 of 24
    📄 Scraping page 6 of 24
    📄 Scraping page 7 of 24
    📄 Scraping page 8 of 24
    📄 Scraping page 9 of 24
    📄 Scraping page 10 of 24
    📄 Scraping page 11 of 24
    📄 Scraping page 12 of 24
    📄 Scraping page 13 of 24
    📄 Scraping page 14 of 24
    📄 Scraping page 15 of 24
    📄 Scraping page 16 of 24
    📄 Scraping page 17 of 24
    📄 Scraping page 18 of 24
    📄 Scraping page 19 of 24
    📄 Scraping page 20 of 24
    📄 Scraping page 21 of 24
    📄 Scraping page 22 of 24
    📄 Scraping page 23 of 24
    📄 Scraping page 24 of 24
  🔤 Scraping letter: b (19 pages, 455 entries)
    📄 Scraping page 1 of 19
    📄 Scraping page 2 of 19
    📄 Scraping page 3 of 19
    📄 Scraping page 4 of 19
    📄 Scraping page 5 of 19
    📄 Scraping page 6 of 19
    📄 Scraping page 7 o

In [69]:
try:
    # Load the JSON file
    with open("ngiemboon_dictionary.json", 'r', encoding='utf-8') as f:
        dataFinal = json.load(f)
    # Print the total number of entries
    print(f"Total entries in the JSON file: {len(dataFinal)}")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: The file '{file_path}' is not a valid JSON file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Total entries in the JSON file: 9774


In [70]:
def find_entry(word):
    for entry in dataFinal:
        if entry["word"].lower() == word.lower():
            return entry
    return "Word not found"

In [77]:
# Example usage
word = "Bangang"
result = find_entry(word)
print(result)

{'Current Language Category': 'French', 'word': 'Bangang', 'Ngienmboon Language word': 'Ngǎŋ', 'Example Usage': 'Ngǎŋn'}
